<a href="https://colab.research.google.com/github/beyerly/ml_notes/blob/master/book_dep_analysis/book_dep_analysis_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Book genre analysis, based on https://www.kaggle.com/sp1thas/book-depository-dataset/data 
# Colab version, with TPU option
# WIP 

import numpy as np
import pandas as pd
import re
import matplotlib
import os
import pprint
import json

In [2]:
%tensorflow_version 1.14
import tensorflow as tf  # 1.14.0
import tensorflow_hub as hub
from datetime import datetime

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [3]:
!pip install bert-tensorflow
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import run_classifier_with_tfhub

In [0]:
flags = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')   # fix from https://stackoverflow.com/questions/48198770/tensorflow-1-5-0-rc0-error-using-tf-app-flags

In [0]:
# Set to use TPU on GCS or CPU on colab
flags.use_tpu = True

In [0]:
# If using CPU, we get data from Drive
if not flags.use_tpu:
  from google.colab import drive
  drive.mount('/content/drive')

In [0]:
# download dataset from kaggle to drive, only do once
tf.app.flags.DEFINE_bool('data_on_drive', True,"Kaggle; dataset already downloaded on Drive")
if not flags.data_on_drive:
  from google.colab import files
  files.upload()  #this will prompt you to upload the kaggle.json

  # install Kaggle API
  !pip install -q kaggle
  !mkdir -p ~/.kaggle
  !cp kaggle.json ~/.kaggle/
  !ls ~/.kaggle
  !chmod 600 /root/.kaggle/kaggle.json  # set permission

  # dowload kaggle API, get key from kaggle account
  !kaggle datasets download -d sp1thas/book-depository-dataset -p /content/drive/My\ Drive/datasets
  !kaggle datasets list -s book
  os.chdir('drive/My Drive/datasets')  #change dir
  !mkdir book-depository-dataset  
  !unzip -q book-depository-dataset.zip -d book-depository-dataset/
  os.chdir('../../')  #change dir

In [0]:
# import dataset
ds = pd.read_csv('drive/My Drive/datasets/book-depository-dataset/dataset.csv', usecols=['authors', 'bestsellers-rank', 'categories', 'description', 'for-ages', 'id', 'lang', 'rating-avg', 'rating-count', 'title'])


In [9]:
ds.head(10)

,authors,bestsellers-rank,categories,description,for-ages,id,lang,rating-avg,rating-count,title
0,"[1, 2]",2896,"[235, 3386, 736, 763]",________________________________,NaN,9781787465145,en,4.04,3784.0,Journey Into Darkness
1,[3],51517,"[360, 2632]",A fake marriage is the last thing he wants . ....,NaN,9781946510402,en,4.49,211.0,The Mercenary : Order of the Broken Blade
2,[4],162985,"[1703, 2771, 2818, 3097]","Once the dust has settled, you'll need to know...",NaN,9781612124568,en,2.96,139.0,100 Skills You'll Need for the End of the Worl...
3,[5],41520,"[819, 3364, 1853, 2977]",The Daily Mail and the Spectator Book of the Y...,NaN,9781786487155,en,4.21,158.0,How to Land a Plane
4,[6],3885,"[1694, 1703, 2818]","Easy, do-able, down to earth ideas and suggest...",NaN,9781472969125,en,NaN,NaN,The Sustainable(ish) Living Guide : Everything...
5,[7],75224,"[1843, 2967, 2969]",Mini celebrates 60 amazing years of this iconi...,NaN,9780760363997,en,4.75,4.0,Mini : 60 Years
6,"[8, 9]",368836,"[1520, 2771, 3019, 3074, 3078]","Dynamics, motion, and sensation are karate's c...",NaN,9781594394591,en,4.48,27.0,Karate Science : Dynamic Movement
7,[10],"108,369","[1853, 2977, 3023]",Brian Cosgrove's classic introduction to the w...,NaN,9781847975096,en,4.38,26.0,Microlight Pilot's Handbook
8,"[11, 12]",49553,"[649, 2771, 3019, 3074]",BECAUSE NOT ALL KRAV MAGA IS THE SAME(R) This ...,NaN,9781594396816,en,4.00,1.0,Krav Maga Combatives : Maximum Effect
9,"[13, 14]","128,070","[1725, 1844, 2969, 2973]","Haynes offers the best coverage for cars, truc...",NaN,9781563924699,en,3.92,13.0,"Dodge Caravan, Plymouth Voyager & Chrysler Tow..."


In [0]:
cat = pd.read_csv('drive/My Drive/datasets/book-depository-dataset/categories.csv')

In [11]:
cat.head(5)

,category_id,category_name
0,1998,.Net Programming
1,176,20th Century & Contemporary Classical Music
2,3291,20th Century & Contemporary Classical Music
3,2659,20th Century History: C 1900 To C 2000
4,2661,21st Century History: From C 2000 -


In [0]:
def findBook(title, df):
    resDf = pd.DataFrame()
    for idx, i in df['title'].str.contains(title, case=False).iteritems():
        if i:
            genres = ""
            res = df['categories'][idx].strip('][').split(', ') 
            for c in res:
                genres += cat.loc[cat['category_id']==int(c)]['category_name'].values[0] + ", "
            resDf = resDf.append(pd.DataFrame([[df['title'][idx], genres]], columns=['title', 'genre']), ignore_index=True)
    return resDf

def findGenre(genre, df):
    resDf = pd.DataFrame()
    catList = []
    catName = []
    for idx, i in cat['category_name'].str.contains(genre, case=False).iteritems():
        if i:
            catList.append((cat['category_id'][idx],cat['category_name'][idx]))
    for idx, i in df['categories'].iteritems():
        for c in catList:
            if str(c[0]) in i.strip('][').split(', '):
                resDf = resDf.append(pd.DataFrame([[df['title'][idx], c[1]]], columns=['title', 'genre']), ignore_index=True)
    return resDf

def printGenre(category):
    print(cat[cat['category_id']==category])


In [0]:
# q=findGenre("Memoirs",ds)
# q.head(10)

In [0]:
# q=findGenre("Science",ds)
# q

In [0]:
# q = findBook('galaxy', ds)
# q.head(100)

In [0]:
def createGenreHistogram(ds):
    d={}
    for idx, i in ds['title'].iteritems():
        res = ds['categories'][idx].strip('][').split(', ') 
        for c in res:
            if len(c)>0:
                if c in d.keys():
                    d[c]+=1
                else:
                    d[c]=1
    f = {}
    for i in d.keys():
        ct = cat.loc[cat['category_id']==int(i)]['category_name'].values
        if len(ct)>0:
            f[ct[0]] = d[i]

    return pd.DataFrame.from_dict({'c':list(f.keys()), 'r':list(f.values())})


In [0]:
# catHist = createGenreHistogram(ds.sample(1000))

In [0]:
# catHist.plot.bar(x='c',y='r', figsize=(50,10))

In [0]:
# catHist = createGenreHistogram(ds)

In [0]:
# catHist

In [0]:
# ds[0:100].hist(column='rating-avg')

In [0]:
# Predict whether book is of any of the 'Science' based genres based on title

In [0]:
science = cat['category_name'].str.contains('science', case=False)==True

In [24]:
catScience = cat[science]
catScience.head(10)

,category_id,category_name
57,1877,Agricultural Science
205,229,"Autobiography: Science, Technology & Engineering"
272,228,"Biography: Science, Technology & Engineering"
275,1593,"Biology, Life Sciences"
298,1622,Botany & Plant Sciences
463,353,Classic Science Fiction
464,2627,Classic Science Fiction
534,2056,Computer Science
779,1652,Earth Sciences
780,1651,"Earth Sciences, Geography, Environment, Planning"


In [0]:
# !pip uninstall bert-for-tf2

In [26]:
# if using CPU, we use Drive for model output

if not flags.use_tpu:
  OUTPUT_DIR = 'drive/My Drive/colab_output/output'#@param {type:"string"}
  tf.gfile.MakeDirs(OUTPUT_DIR)
else:
  # using TPU, so we must use GCS bucket for data, TPU cannot access Drive
  from google.colab import auth
  auth.authenticate_user()
  TASK = 'MRPC' #@param {type:"string"}
  BUCKET = 'bookdep_bert' #@param {type:"string"}
  assert BUCKET, 'Must specify an existing GCS bucket name'
  OUTPUT_DIR = 'gs://{}/bert-tfhub/models/{}'.format(BUCKET, TASK)
  tf.gfile.MakeDirs(OUTPUT_DIR)
  os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

print('***** Model output directory: {} *****'.format(OUTPUT_DIR))
label_list = [0, 1]

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

***** Model output directory: gs://bookdep_bert/bert-tfhub/models/MRPC *****


In [0]:
def isScienceGenre(categories, catScience):
    res = categories.strip('][').split(', ') 
    for c in res:
        if c:
            if int(c) in catScience['category_id'].values:
                return 1
    return 0

#p = ds.sample(10000)
ds['isScience'] =  ds.apply(lambda x: isScienceGenre(x['categories'], catScience),axis=1)

In [0]:
# ds[ds['isScience']==1].head(50)
# ds[ds['isScience']==1].loc[204]

In [29]:
dsTrain = ds.sample(5000)
dsTest = ds.sample(1000)
dsTrain.columns

Index(['authors', 'bestsellers-rank', 'categories', 'description', 'for-ages',
       'id', 'lang', 'rating-avg', 'rating-count', 'title', 'isScience'],
      dtype='object')

In [0]:
train_InputExamples = dsTrain.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['title'], 
                                                                   text_b = None, 
                                                                   label = x['isScience']), axis = 1)
test_InputExamples = dsTest.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['title'], 
                                                                   text_b = None, 
                                                                   label = x['isScience']), axis = 1)


In [31]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [32]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 5000


INFO:tensorflow:Writing example 0 of 5000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):

    bert_module = hub.Module(BERT_MODEL_HUB, trainable=True)
    bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
    bert_outputs = bert_module(inputs=bert_inputs, signature="tokens", as_dict=True)

    output_layer = bert_outputs["pooled_output"]
    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for politeness data.
    output_weights = tf.get_variable("output_weights", [num_labels, hidden_size],
                                     initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable("output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):

        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        
        # tf.summary.scalar('loss', loss) # https://github.com/tensorflow/tensorflow/issues/37568
        
        return (loss, predicted_labels, log_probs)


In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
        # TRAIN and EVAL
        if not is_predicting:

            (loss, predicted_labels, log_probs) = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            train_op = bert.optimization.create_optimizer(
                loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=flags.use_tpu)

            # Calculate evaluation metrics. 
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                f1_score = tf.contrib.metrics.f1_score(label_ids, predicted_labels)
                auc = tf.metrics.auc(label_ids, predicted_labels)
                recall = tf.metrics.recall(label_ids, predicted_labels)
                precision = tf.metrics.precision(label_ids, predicted_labels) 
                true_pos = tf.metrics.true_positives(label_ids, predicted_labels)
                true_neg = tf.metrics.true_negatives(label_ids, predicted_labels)   
                false_pos = tf.metrics.false_positives(label_ids, predicted_labels)  
                false_neg = tf.metrics.false_negatives(label_ids, predicted_labels)

                return {
                    "eval_accuracy": accuracy,
                    "f1_score": f1_score,
                    "auc": auc,
                    "precision": precision,
                    "recall": recall,
                    "true_positives": true_pos,
                    "true_negatives": true_neg,
                    "false_positives": false_pos,
                    "false_negatives": false_neg}

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

        predictions = {'probabilities': log_probs,
                       'labels': predicted_labels}
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn


In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [38]:
# Specify outpit directory and number of checkpoint steps to save
# unused, use TPU runconfig


if not flags.use_tpu:
  run_config = tf.estimator.RunConfig(
      model_dir=OUTPUT_DIR,
      save_summary_steps=SAVE_SUMMARY_STEPS,
      save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)
else:
  # Setup TPU related config
  if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
  else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)
    with tf.Session(tpu_address) as session:
      devices = session.list_devices()
      # Upload credentials to TPU.
      with open('/content/adc.json', 'r') as f:
        auth_info = json.load(f)
        tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
        # Now credentials are set for all future sessions on this TPU.
    print('TPU devices:')
    pprint.pprint(devices)

  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu_address)
  NUM_TPU_CORES = 8
  ITERATIONS_PER_LOOP = 1000

  run_config = tf.contrib.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      model_dir=OUTPUT_DIR,
      save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
      tpu_config=tf.contrib.tpu.TPUConfig(
          iterations_per_loop=ITERATIONS_PER_LOOP,
          num_shards=NUM_TPU_CORES,
          per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

TPU address is grpc://10.68.129.146:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 1110193118053543326),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 16602744076209716013),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 1115558502869672367),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 3804215398755453828),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 3368243122158734153),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 749310827435301058),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 13431808278741088198),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 10918670715806564089),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 1320417072514

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

In [40]:
if not flags.use_tpu:
  estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    config=run_config,
    params={"batch_size": BATCH_SIZE})
else:
  estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=flags.use_tpu,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=BATCH_SIZE,
    eval_batch_size=BATCH_SIZE,
    predict_batch_size=BATCH_SIZE,
  )

INFO:tensorflow:Using config: {'_model_dir': 'gs://bookdep_bert/bert-tfhub/models/MRPC', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.68.129.146:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f87e37285f8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.68.129.146:8470', '_evaluation_master': 'grpc://10.68.129.146:8470', '_is_chief': True, '_num_

INFO:tensorflow:Using config: {'_model_dir': 'gs://bookdep_bert/bert-tfhub/models/MRPC', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.68.129.146:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f87e37285f8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.68.129.146:8470', '_evaluation_master': 'grpc://10.68.129.146:8470', '_is_chief': True, '_num_

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


In [0]:
# Create an input function for training
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)    # https://stackoverflow.com/questions/58029896/bert-fine-tuning-with-estimators-on-tpus-on-colab-typeerror-unsupported-operand

    

In [42]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Querying Tensorflow master (grpc://10.68.129.146:8470) for TPU system metadata.


INFO:tensorflow:Querying Tensorflow master (grpc://10.68.129.146:8470) for TPU system metadata.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


...

INFO:tensorflow:Loss for final step: 0.00034270954.


INFO:tensorflow:Loss for final step: 0.00034270954.


INFO:tensorflow:training_loop marked as finished


INFO:tensorflow:training_loop marked as finished


Training took time  0:02:05.439949


In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

In [0]:
def getPrediction(in_sentences):
  labels = ["No", "Yes"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
pred_sentences = [
  "Astronomy for beginners",
  "Intro biochemistry",
  "pride and prejudice",
  "computer science fundamentals",
  "Chronicles of Narnia",
  "the old man and the sea",
  "physics for computer science students"
]

In [0]:
predictions = getPrediction(pred_sentences)

In [0]:
predictions